In [1]:
using ValkyrieLCMSim
using RigidBodySim
using ValkyrieRobot
using ValkyrieRobot.BipedControlUtil
using RigidBodyDynamics.Contact

In [15]:
val = Valkyrie()
mechanism = val.mechanism
# remove_fixed_tree_joints!(mechanism);

In [3]:
# add environment
rootframe = root_frame(mechanism)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism, ground);

In [4]:
# Create visualizer
any_open_visualizer_windows() || (new_visualizer_window(); sleep(1));
vis = Visualizer()[:valkyrie]
setgeometry!(vis, mechanism, parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]))

In [5]:
feet = Dict(:left => findbody(mechanism, "leftFoot"), :right => findbody(mechanism, "rightFoot"))
hands = Dict(:left => findbody(mechanism, "leftPalm"), :right => findbody(mechanism, "rightPalm"))
robot_info = HumanoidRobotInfo(mechanism, feet, hands);

In [6]:
state = MechanismState(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [7]:
function initialize(state::MechanismState, val::Valkyrie, vis::Visualizer)
    zero!(state)
    kneebend = 1.1
    hipbendextra = 0.1
    for side in instances(Side)
        set_configuration!(state, val.knees[side], [kneebend])
        set_configuration!(state, val.hippitches[side], [-kneebend / 2 + hipbendextra])
        set_configuration!(state, val.anklepitches[side], [-kneebend / 2 - hipbendextra])
    end
    set_configuration!(state, val.basejoint, [1; 0; 0; 0; 0; 0; 1.05]) # TODO
    settransform!(vis, state)
end

initialize (generic function with 1 method)

In [8]:
function periodic_lcm_controller(robot_info::HumanoidRobotInfo{T}, Δt::Number) where T
    lcmcontroller = LCMController(robot_info)
    controller = PeriodicController{T}(num_velocities(robot_info.mechanism), Δt, lcmcontroller)
    initialize_lcmcontroller = (c, t, u, integrator) -> ValkyrieLCMSim.initialize!(lcmcontroller)
    periodic_control_callback = PeriodicCallback(controller, initialize = initialize_lcmcontroller)
    controller, periodic_control_callback
end

periodic_lcm_controller (generic function with 1 method)

In [13]:
controller, periodic_control_callback = periodic_lcm_controller(robot_info, 1 / 300)
initialize(state, val, vis)
final_time = Inf
problem = ODEProblem(state, (0., final_time), controller)
callbacks = CallbackSet(periodic_control_callback, CallbackSet(vis, state))
sol = solve(problem, Tsit5(), abs_tol = 1e-10, dt = 0.05, callback = callbacks);

ERROR (unhandled task failure): cannot resize array with shared data
Stacktrace:
 [1] empty! at ./array.jl:1042 [inlined]
 [2] close(::BufferedStreams.BufferedInputStream{BufferedStreams.EmptyStream}) at /home/twan/code/RigidBodyDynamics/v0.6/BufferedStreams/src/bufferedinputstream.jl:367
 [3] handle_robot_command_msg(::ValkyrieLCMSim.LCMController, ::Array{UInt8,1}) at /home/twan/code/RigidBodyDynamics/v0.6/ValkyrieLCMSim/src/control.jl:52
 [4] onresponse(::LCMCore.RecvBuf, ::Ptr{UInt8}, ::LCMCore.SubscriptionOptions{Void,ValkyrieLCMSim.##28#31{ValkyrieLCMSim.LCMController}}) at /home/twan/code/RigidBodyDynamics/v0.6/LCMCore/src/LCMCore.jl:177
 [5] handle(::LCMCore.LCM) at /home/twan/code/RigidBodyDynamics/v0.6/LCMCore/src/LCMCore.jl:219
 [6] macro expansion at /home/twan/code/RigidBodyDynamics/v0.6/ValkyrieLCMSim/src/control.jl:36 [inlined]
 [7] (::ValkyrieLCMSim.##29#32{LCMCore.LCM})() at ./task.jl:335


LoadError: [91mKeyError: key "waistLeftActuator" not found[39m

In [10]:
# animate(vis, state, sol)

In [11]:
# function testfunc(controller, τ, t, state, n)
#     for i = 1 : n
#         controller(τ, t, state)
#     end
# end

# testfunc(controller, τ, t, state, 1)
# Profile.clear_malloc_data()
# @profile testfunc(controller, τ, t, state, 100000)

# using ProfileView
# ProfileView.view()